In [30]:
## Use this codes to lookup the diagnosis codes, lab proc, medication
## Author: Ahmed Abdelhameed
## Date: July 06, 2022

import pandas as pd
import numpy as np
import csv
from collections import OrderedDict
import re
#import path as path

# Load code descriptions

In [31]:
# import the whole lookup table to check merging
import pandas as pd
raw = pd.read_csv('/ctao_shared_data/Optum_lookup/lu_diagnosis.csv', delimiter='\t', low_memory=False)
raw

,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,gdr_spec_cd,mdc_cd_desc,mdc_code,icd_ver_cd
0,23297,UNKNOWN DIAGNOSIS,232,UNKNOWN DIAGNOSIS,2329,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
1,84110,UNKNOWN DIAGNOSIS,841,UNKNOWN DIAGNOSIS,8411,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
2,23663,UNKNOWN DIAGNOSIS,236,UNKNOWN DIAGNOSIS,2366,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
3,91120,UNKNOWN DIAGNOSIS,911,UNKNOWN DIAGNOSIS,9112,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
4,CP590,UNKNOWN DIAGNOSIS,CP5,UNKNOWN DIAGNOSIS,CP59,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,9
...,...,...,...,...,...,...,...,...,...,...
316774,S72433C,DSPL FX MED C UNS FEM INIT OPEN III,S72,FRACTURE OF FEMUR,S724,FRACTURE OF LOWER END OF FEMUR,NaN,DISEASES & DISORDERS OF THE MUSCULOSKELETAL SYSTE,08,10
316775,I0741,UNKNOWN DIAGNOSIS,I07,UNKNOWN DIAGNOSIS,I074,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
316776,K13159,UNKNOWN DIAGNOSIS,K13,UNKNOWN DIAGNOSIS,K131,UNKNOWN DIAGNOSIS,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10
316777,V80730,ANIMAL-RIDER INJ COLL STREETCAR,V80,ANML-RIDR/OCC ANML VEH INJ TRNS ACC,V807,ANML-RIDR/OCC ANML VEH INJ COLL NMV,NaN,UNKNOWN MAJOR DIAGNOSTIC CATEGORY,00,10


In [32]:
#raw[raw["diag_cd"] == "L851"]
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316779 entries, 0 to 316778
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   diag_cd         316777 non-null  object
 1   diag_desc       316779 non-null  object
 2   diag_fst3_cd    316696 non-null  object
 3   diag_fst3_desc  316779 non-null  object
 4   diag_fst4_cd    306578 non-null  object
 5   diag_fst4_desc  306411 non-null  object
 6   gdr_spec_cd     6108 non-null    object
 7   mdc_cd_desc     316779 non-null  object
 8   mdc_code        316779 non-null  object
 9   icd_ver_cd      316779 non-null  object
dtypes: object(10)
memory usage: 24.2+ MB


## Filter Some columns

In [33]:
lu = raw[['diag_cd','diag_desc','diag_fst3_cd','diag_fst3_desc', 'diag_fst4_cd','diag_fst4_desc','icd_ver_cd']]
lu = lu.loc[lu.diag_desc!='UNKNOWN DIAGNOSIS']
lu = lu.drop_duplicates().reset_index(drop=True)
lu.isnull().sum()

diag_cd              0
diag_desc            0
diag_fst3_cd         0
diag_fst3_desc       0
diag_fst4_cd      2949
diag_fst4_desc    3138
icd_ver_cd           0
dtype: int64

In [34]:
lu

,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
0,4275,CARDIAC ARREST,427,CARDIAC DYSRHYTHMIAS,4275,CARDIAC ARREST,9
1,9649,POISN UNS AGT AFFCT BLD CONSTITUNTS,964,POIS-BLOOD AFFECTING AGT,9649,POISONING-BLOOD AGT NOS,9
2,X942XXD,ASSAULT BY MACHINE GUN SUBSQT ENC,X94,ASLT RIFLE SHOTGUN LARGR FIRARM D/C,X942,ASSAULT BY MACHINE GUN,10
3,W551,CONTACT WITH HORSE,W55,CONTACT WITH OTHER MAMMALS,W551,CONTACT WITH HORSE,10
4,S82044D,NDSPL COMMNT FX RT PAT SUB CLOS RTN,S82,FRACTURE LOWER LEG INCLUDING ANKLE,S820,FRACTURE OF PATELLA,10
...,...,...,...,...,...,...,...
112799,T24022S,BURN UNS DEGREE LT KNEE SEQUELA,T24,BURN CORROSION LOW LIMB NO ANK FOOT,T240,BURN UNS DEG LOW LIMB NO ANKLE FOOT,10
112800,S52592A,OTH FX LOW LT RADIUS INIT CLOS FX,S52,FRACTURE OF FOREARM,S525,FRACTURE OF LOWER END OF RADIUS,10
112801,S4239,OTHER FRACTURE OF SHAFT OF HUMERUS,S42,FRACTURE OF SHOULDER AND UPPER ARM,S423,FRACTURE OF SHAFT OF HUMERUS,10
112802,S72433C,DSPL FX MED C UNS FEM INIT OPEN III,S72,FRACTURE OF FEMUR,S724,FRACTURE OF LOWER END OF FEMUR,10


In [35]:
lu[lu["diag_cd"] == 'Z944']

,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
49438,Z944,LIVER TRANSPLANT STATUS,Z94,TRANSPLANTED ORGAN & TISSUE STATUS,Z944,LIVER TRANSPLANT STATUS,10


In [36]:
lu.icd_ver_cd.value_counts()

10            94688
9             18115
ICD_VER_CD        1
Name: icd_ver_cd, dtype: int64

In [37]:
my_row = lu[lu.icd_ver_cd=="ICD_VER_CD"].index

In [38]:
lu = lu.drop(my_row)

In [39]:
lu['icd_ver_cd']=lu['icd_ver_cd'].astype(int)

## Diagnosis 

In [40]:
f = "final_diag_all"
path = '/ctao_shared_data/Liver_optum/outputs/'
diagnosis = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)

In [41]:
diagnosis

,patid,fst_dt,icd_flag,diag
0,560499200082764,2017-09-07,10,Z944
1,560499200082764,2017-12-21,10,Z4823
2,560499200082764,2017-12-21,10,Z944
3,560499200082764,2017-12-21,10,E7223
4,560499200082764,2017-12-21,10,Z48298
...,...,...,...,...
16016911,560499899999890,2015-02-11,9,78079
16016912,560499899999890,2015-02-11,9,78605
16016913,560499899999890,2015-02-11,9,3569
16016914,560499899999890,2015-02-11,9,V5869


In [42]:
diagnosis.icd_flag.value_counts()

10    8260060
9     7756856
Name: icd_flag, dtype: int64

## Identify ICD-9 and ICD-10

In [43]:
diagnosis['modified_diag'] = np.where((diagnosis['icd_flag']==9), 'D_9_' + diagnosis['diag'].replace('.', ''), 'D_10_' + diagnosis['diag'].replace('.', ''))

In [44]:
diagnosis

,patid,fst_dt,icd_flag,diag,modified_diag
0,560499200082764,2017-09-07,10,Z944,D_10_Z944
1,560499200082764,2017-12-21,10,Z4823,D_10_Z4823
2,560499200082764,2017-12-21,10,Z944,D_10_Z944
3,560499200082764,2017-12-21,10,E7223,D_10_E7223
4,560499200082764,2017-12-21,10,Z48298,D_10_Z48298
...,...,...,...,...,...
16016911,560499899999890,2015-02-11,9,78079,D_9_78079
16016912,560499899999890,2015-02-11,9,78605,D_9_78605
16016913,560499899999890,2015-02-11,9,3569,D_9_3569
16016914,560499899999890,2015-02-11,9,V5869,D_9_V5869


In [45]:
#diagnosis[(diagnosis.diag == '04100') & (diagnosis.icd_flag == 9)]

In [46]:
col = ['icd_flag', 'diag', 'modified_diag']
diagnosis= diagnosis[col]
diagnosis

,icd_flag,diag,modified_diag
0,10,Z944,D_10_Z944
1,10,Z4823,D_10_Z4823
2,10,Z944,D_10_Z944
3,10,E7223,D_10_E7223
4,10,Z48298,D_10_Z48298
...,...,...,...
16016911,9,78079,D_9_78079
16016912,9,78605,D_9_78605
16016913,9,3569,D_9_3569
16016914,9,V5869,D_9_V5869


In [47]:
diag_filtered = diagnosis.drop_duplicates()
dups = diag_filtered.loc[diag_filtered.diag.duplicated(), :]
dup_list = dups['diag'].tolist()

In [48]:
len(dup_list)

368

In [49]:
dups

,icd_flag,diag,modified_diag
26587,9,E8801,D_9_E8801
122739,9,L851,D_9_L851
122743,9,M79673,D_9_M79673
122744,9,L609,D_9_L609
134832,9,V9999,D_9_V9999
...,...,...,...
15805180,9,S81802D,D_9_S81802D
15805183,9,T66XXXA,D_9_T66XXXA
15857290,10,27652,D_10_27652
15857292,10,5119,D_10_5119


In [50]:
diag_filtered [diag_filtered.diag =="5119"]

,icd_flag,diag,modified_diag
8014,9,5119,D_9_5119
15857292,10,5119,D_10_5119


In [51]:
diag_filtered.modified_diag.nunique()

33214

In [52]:
diag_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33214 entries, 0 to 16011431
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   icd_flag       33214 non-null  int64 
 1   diag           33214 non-null  object
 2   modified_diag  33214 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [53]:
#merged = diag_filtered.merge(lu, left_on='diag', right_on='diag_cd')

In [58]:
merged = diag_filtered.merge(lu, left_on=['diag','icd_flag'], right_on=['diag_cd', 'icd_ver_cd'])

In [20]:
#merged.head(10)

,icd_flag,diag,modified_diag,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
0,10,Z944,D_10_Z944,Z944,LIVER TRANSPLANT STATUS,Z94,TRANSPLANTED ORGAN & TISSUE STATUS,Z944,LIVER TRANSPLANT STATUS,10
1,10,Z4823,D_10_Z4823,Z4823,ENC AFTERCARE FLW LIVER TRANSPLANT,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
2,10,E7223,D_10_E7223,E7223,CITRULLINEMIA,E72,OTH DISORDERS AMINO-ACID METABOLISM,E722,DISORDERS OF UREA CYCLE METABOLISM,10
3,10,Z48298,D_10_Z48298,Z48298,ENC AFTERCARE FLW OTH ORGN TRANSPL,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
4,10,Z23,D_10_Z23,Z23,ENCOUNTER FOR IMMUNIZATION,Z23,ENCOUNTER FOR IMMUNIZATION,NaN,NaN,10
5,10,L570,D_10_L570,L570,ACTINIC KERATOSIS,L57,SKIN CHANGES CHRN EXPS NONIONIZ RAD,L570,ACTINIC KERATOSIS,10
6,10,Z08,D_10_Z08,Z08,ENC F/U EX AFTR CMPL TX MALIG NEOPL,Z08,ENC F/U EX AFTR CMPL TX MALIG NEOPL,NaN,NaN,10
7,10,L814,D_10_L814,L814,OTHER MELANIN HYPERPIGMENTATION,L81,OTHER DISORDERS OF PIGMENTATION,L814,OTHER MELANIN HYPERPIGMENTATION,10
8,10,L821,D_10_L821,L821,OTHER SEBORRHEIC KERATOSIS,L82,SEBORRHEIC KERATOSIS,L821,OTHER SEBORRHEIC KERATOSIS,10
9,10,Z85828,D_10_Z85828,Z85828,PERSONAL HX OTH MALIG NEOPLASM SKIN,Z85,PERSONAL HISTORY MALIGNANT NEOPLASM,Z858,PERS HX MALIG NEOPLSM OTH ORGN&SYS,10


In [59]:
merged

,icd_flag,diag,modified_diag,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
0,10,Z944,D_10_Z944,Z944,LIVER TRANSPLANT STATUS,Z94,TRANSPLANTED ORGAN & TISSUE STATUS,Z944,LIVER TRANSPLANT STATUS,10
1,10,Z4823,D_10_Z4823,Z4823,ENC AFTERCARE FLW LIVER TRANSPLANT,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
2,10,E7223,D_10_E7223,E7223,CITRULLINEMIA,E72,OTH DISORDERS AMINO-ACID METABOLISM,E722,DISORDERS OF UREA CYCLE METABOLISM,10
3,10,Z48298,D_10_Z48298,Z48298,ENC AFTERCARE FLW OTH ORGN TRANSPL,Z48,ENC OTHER POSTPROCEDURAL AFTERCARE,Z482,ENC AFTERCARE FLW ORGAN TRANSPLANT,10
4,10,Z23,D_10_Z23,Z23,ENCOUNTER FOR IMMUNIZATION,Z23,ENCOUNTER FOR IMMUNIZATION,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...
31998,9,3720,D_9_3720,3720,ACUTE CONJUNCTIVITIS,372,DISORDERS OF CONJUNCTIVA,3720,ACUTE CONJUNCTIVITIS,9
31999,10,T17298A,D_10_T17298A,T17298A,OTH FOREIGN OBJ PHAR OTH INJ INIT,T17,FOREIGN BODY IN RESPIRATORY TRACT,T172,FOREIGN BODY IN PHARYNX,10
32000,10,T84296S,D_10_T84296S,T84296S,OTH MECH COMP IF DEVC VERTEBRAE SEQ,T84,COMP INTRL ORTHO PROS DEV IMPL GFT,T842,MECH COMP INTERNAL FIX OTH BONES,10
32001,9,29100,D_9_29100,29100,ALCHL WTHDRWL DELIRIUM,291,ALCOHOLIC PSYCHOSES,2910,DELIRIUM TREMENS,9


In [60]:
merged[merged['diag'] == "E8801"]

,icd_flag,diag,modified_diag,diag_cd,diag_desc,diag_fst3_cd,diag_fst3_desc,diag_fst4_cd,diag_fst4_desc,icd_ver_cd
201,10,E8801,D_10_E8801,E8801,ALPHA-1-ANTITRYPSIN DEFICIENCY,E88,OTHER & UNS METABOLIC DISORDERS,E880,DISORDERS PLASMA-PROTEIN METAB NEC,10
2408,9,E8801,D_9_E8801,E8801,ACCIDENTAL FALL/FROM SIDEWALK CURB,E88,FALL,E880,FALL ON STAIRS OR STEPS,9


In [61]:
col = [ 'diag', 'modified_diag', 'diag_desc']
merged = merged[col]
diag_final_desc = merged.drop_duplicates()
diag_final_desc

,diag,modified_diag,diag_desc
0,Z944,D_10_Z944,LIVER TRANSPLANT STATUS
1,Z4823,D_10_Z4823,ENC AFTERCARE FLW LIVER TRANSPLANT
2,E7223,D_10_E7223,CITRULLINEMIA
3,Z48298,D_10_Z48298,ENC AFTERCARE FLW OTH ORGN TRANSPL
4,Z23,D_10_Z23,ENCOUNTER FOR IMMUNIZATION
...,...,...,...
31998,3720,D_9_3720,ACUTE CONJUNCTIVITIS
31999,T17298A,D_10_T17298A,OTH FOREIGN OBJ PHAR OTH INJ INIT
32000,T84296S,D_10_T84296S,OTH MECH COMP IF DEVC VERTEBRAE SEQ
32001,29100,D_9_29100,ALCHL WTHDRWL DELIRIUM


In [62]:
diag_final_desc.modified_diag.nunique()

32003

In [63]:
diag_final_desc[diag_final_desc.diag=='E8801']

,diag,modified_diag,diag_desc
201,E8801,D_10_E8801,ALPHA-1-ANTITRYPSIN DEFICIENCY
2408,E8801,D_9_E8801,ACCIDENTAL FALL/FROM SIDEWALK CURB


In [64]:
diag_final_desc['cat'] = 'diag'
diag_final_desc= diag_final_desc.drop(columns='diag')
diag_final_desc = diag_final_desc.rename(columns = {'modified_diag': 'DIAGNOSIS_ID', 'diag_desc': 'description'})
col = ['DIAGNOSIS_ID','cat','description']
diag_final_desc = diag_final_desc[col]
diag_final_desc

,DIAGNOSIS_ID,cat,description
0,D_10_Z944,diag,LIVER TRANSPLANT STATUS
1,D_10_Z4823,diag,ENC AFTERCARE FLW LIVER TRANSPLANT
2,D_10_E7223,diag,CITRULLINEMIA
3,D_10_Z48298,diag,ENC AFTERCARE FLW OTH ORGN TRANSPL
4,D_10_Z23,diag,ENCOUNTER FOR IMMUNIZATION
...,...,...,...
31998,D_9_3720,diag,ACUTE CONJUNCTIVITIS
31999,D_10_T17298A,diag,OTH FOREIGN OBJ PHAR OTH INJ INIT
32000,D_10_T84296S,diag,OTH MECH COMP IF DEVC VERTEBRAE SEQ
32001,D_9_29100,diag,ALCHL WTHDRWL DELIRIUM


In [184]:
#desc1 = pd.read_csv('desc1.csv')

In [ ]:
#desc1

In [203]:
#desc1[~desc1.isin(diag_final_desc)].dropna()
# common = desc1.merge(diag_final_desc,on=['DIAGNOSIS_ID','cat','description'])
# print(common)
# desc1[(~desc1.DIAGNOSIS_ID.isin(common.DIAGNOSIS_ID))&(~desc1.cat.isin(common.cat))&(~desc1.description.isin(common.description))]

# df = pd.concat([desc1,diag_final_desc])
# df.drop_duplicates(keep=False)

,DIAGNOSIS_ID,cat,description
139,D_10_E8342,diag,OTH FALL WATER TRANSPORT-CREW
146,D_10_E872,diag,FAIL STERILE PRECAUTION DURING PROC
159,D_10_E871,diag,FOREIGN OBJECT LT BODY DURING PROC
165,D_10_E861,diag,ACC POISN CLEAN PAINT VARNISH
205,D_10_E8801,diag,ACCIDENTAL FALL/FROM SIDEWALK CURB
...,...,...,...
31992,D_9_75245,diag,VAGINAL AGENESIS
31997,D_9_87339,diag,OPEN WOUND NOSE MULTIPLE SITES COMP
31998,D_9_3720,diag,ACUTE CONJUNCTIVITIS
32001,D_9_29100,diag,ALCHL WTHDRWL DELIRIUM


In [35]:
diag_final_desc[diag_final_desc.DIAGNOSIS_ID=='D_10_I8381']

,DIAGNOSIS_ID,cat,description
31517,D_10_I8381,diag,VARICOSE VEINS LOWER EXTREM W/PAIN


In [195]:
#desc1[desc1.DIAGNOSIS_ID=='D_10_I8381']

,DIAGNOSIS_ID,cat,description
32005,D_10_I8381,diag,VARICOSE VEINS LOWER EXTREM W/PAIN


In [179]:
#print i

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i)? (1927283887.py, line 1)

In [ ]:
#dups2 = diag_final_desc.loc[diag_final_desc.modified_diag	.duplicated(keep=False), :]
#dups2
#dups2.to_csv('dups2.csv')

In [ ]:
#diag_final_desc['cat'] = 'diag'
#diag_final_desc= diag_final_desc.drop(columns='diag')
#diag_final_desc = diag_final_desc.rename(columns = {'modified_diag': 'DIAGNOSIS_ID', 'diag_desc': 'description'})
#col = ['DIAGNOSIS_ID','cat','description']
#diag_final_desc = diag_final_desc[col]
#diag_final_desc

# Procedures

In [65]:
f ='final_proc_all'
path = '/ctao_shared_data/Liver_optum/outputs/'
proc = pd.read_csv(path+f+'.csv', delimiter = '\t', low_memory = False)
proc

,patid,fst_dt,proc,icd_flag
0,560499200082764,2017-09-07,83735,10
1,560499200082764,2017-09-07,85610,10
2,560499200082764,2017-09-07,82248,10
3,560499200082764,2017-09-07,80053,10
4,560499200082764,2017-09-07,36415,10
...,...,...,...,...
13578877,560499899999890,2015-02-11,99238,9
13578878,560499899999890,2015-02-11,A0425,9
13578879,560499899999890,2015-02-11,E0439,9
13578880,560499899999890,2015-02-11,A0428,9


In [66]:
mod_proc_list = []
for proc_code in proc.proc:
    new_code = 'P_' + 'I_' + proc_code.replace('.', '')
    mod_proc_list.append(new_code)
proc['mod_proc_code'] = mod_proc_list
proc

,patid,fst_dt,proc,icd_flag,mod_proc_code
0,560499200082764,2017-09-07,83735,10,P_I_83735
1,560499200082764,2017-09-07,85610,10,P_I_85610
2,560499200082764,2017-09-07,82248,10,P_I_82248
3,560499200082764,2017-09-07,80053,10,P_I_80053
4,560499200082764,2017-09-07,36415,10,P_I_36415
...,...,...,...,...,...
13578877,560499899999890,2015-02-11,99238,9,P_I_99238
13578878,560499899999890,2015-02-11,A0425,9,P_I_A0425
13578879,560499899999890,2015-02-11,E0439,9,P_I_E0439
13578880,560499899999890,2015-02-11,A0428,9,P_I_A0428


In [67]:
col = ['icd_flag', 'proc', 'mod_proc_code']
proc_filtered = proc[col].drop_duplicates()
proc_filtered 

,icd_flag,proc,mod_proc_code
0,10,83735,P_I_83735
1,10,85610,P_I_85610
2,10,82248,P_I_82248
3,10,80053,P_I_80053
4,10,36415,P_I_36415
...,...,...,...
13573152,10,0DQ10ZZ,P_I_0DQ10ZZ
13573157,10,0CUM07Z,P_I_0CUM07Z
13573200,10,42953,P_I_42953
13575305,9,15933,P_I_15933


In [68]:
# import the whole lookup table to check merging
import pandas as pd
raw = pd.read_csv('/ctao_shared_data/Optum_lookup/lu_procedure.csv', delimiter='\t', low_memory=False)
raw

,category_dtl_cd,category_dtl_code_desc,category_genl_cd,category_genl_code_desc,proc_cd,proc_desc,proc_end_date,proc_typ_cdproc20
0,CATEGORY_DTL_CD,CATEGORY_DTL_CODE_DESC,CATEGORY_GENL_CD,CATEGORY_GENL_CODE_DESC,PROC_CD,PROC_DESC,PROC_END_DATE,PROC_TYP_CD
1,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,0,UNKNOWN PROCEDURE,9999-12-31,UNKN
2,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,2,UNKNOWN PROCEDURE,9999-12-31,UNKN
3,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,0,UNKNOWN PROCEDURE,9999-12-31,UNKN
4,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,1,UNKNOWN PROCEDURE,9999-12-31,UNKN
...,...,...,...,...,...,...,...,...
329853,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZ3,UNKNOWN PROCEDURE,9999-12-31,UNKN
329854,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZ5,UNKNOWN PROCEDURE,9999-12-31,UNKN
329855,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZJ,UNKNOWN PROCEDURE,9999-12-31,UNKN
329856,999,NOT ASSIGNED,16,MISC DIAG THERAPEUTIC,ZZZZX,UNKNOWN PROCEDURE,9999-12-31,UNKN


In [73]:
#lu = raw[['diag_cd','diag_desc','diag_fst3_cd','diag_fst3_desc', 'diag_fst4_cd','diag_fst4_desc','icd_ver_cd']]
lu = raw.loc[raw.proc_desc!='UNKNOWN PROCEDURE']
lu = lu.loc[raw.proc_typ_cdproc20!='PROC_TYP_CD']
lu = lu.loc[raw.proc_typ_cdproc20!='UNKN']
lu = lu.drop_duplicates().reset_index(drop=True)
lu.isnull().sum()

category_dtl_cd            1
category_dtl_code_desc     1
category_genl_cd           1
category_genl_code_desc    1
proc_cd                    0
proc_desc                  1
proc_end_date              0
proc_typ_cdproc20          0
dtype: int64

In [74]:
lu

,category_dtl_cd,category_dtl_code_desc,category_genl_cd,category_genl_code_desc,proc_cd,proc_desc,proc_end_date,proc_typ_cdproc20
0,RDB,RDB,RDB,RDB,.2540,AUDIO DIAGNOSTIC BATTERY,2004-12-31,CPT-4
1,231,OTHER THERAPEUTIC PROCEDURES,16,MISC DIAG THERAPEUTIC,00,PROCEDURES AND INTERVENTIONS NEC,9999-12-31,ICD-9
2,197,OTHER DIAGNOSTIC ULTRASOUND,16,MISC DIAG THERAPEUTIC,000,THERAPEUTIC ULTRASOUND,9999-12-31,ICD-9
3,192,DIAG ULTRASOUND OF HEAD/NECK,16,MISC DIAG THERAPEUTIC,0001,THERAPEUTIC US VESSELS HEAD&NECK,9999-12-31,ICD-9
4,231,OTHER THERAPEUTIC PROCEDURES,16,MISC DIAG THERAPEUTIC,0001F,HEART FAILURE COMPOSITE,9999-12-31,CPT-4
...,...,...,...,...,...,...,...,...
110739,RDB,RDB,RDB,RDB,Z9172,ADAPTIC THEUR ITEMS TOYS,2004-12-31,HCPCS
110740,RDB,RDB,RDB,RDB,Z9173,VAN LIFTS WHEELCHAIR TIE DWN,2004-12-31,HCPCS
110741,RDB,RDB,RDB,RDB,Z9174,ENVIR SAFETY /CONTROLDEVICES,2004-12-31,HCPCS
110742,RDB,RDB,RDB,RDB,Z9175,ALLERGY CONTROL SUPPLIES,2004-12-31,HCPCS


In [75]:
lu[lu['proc_cd']=='93306']

,category_dtl_cd,category_dtl_code_desc,category_genl_cd,category_genl_code_desc,proc_cd,proc_desc,proc_end_date,proc_typ_cdproc20
90398,193,DIAGNOSTIC ULTRASOUND OF HEART,16,MISC DIAG THERAPEUTIC,93306,"TTE W/DOPPLER, COMPLETE",9999-12-31,CPT-4


In [76]:
merged2 = proc_filtered.merge(lu, left_on='proc', right_on='proc_cd')

In [77]:
merged2

,icd_flag,proc,mod_proc_code,category_dtl_cd,category_dtl_code_desc,category_genl_cd,category_genl_code_desc,proc_cd,proc_desc,proc_end_date,proc_typ_cdproc20
0,10,83735,P_I_83735,233,LAB - CHEM & HEMOTOLOGY,16,MISC DIAG THERAPEUTIC,83735,ASSAY OF MAGNESIUM,9999-12-31,CPT-4
1,9,83735,P_I_83735,233,LAB - CHEM & HEMOTOLOGY,16,MISC DIAG THERAPEUTIC,83735,ASSAY OF MAGNESIUM,9999-12-31,CPT-4
2,0,83735,P_I_83735,233,LAB - CHEM & HEMOTOLOGY,16,MISC DIAG THERAPEUTIC,83735,ASSAY OF MAGNESIUM,9999-12-31,CPT-4
3,10,85610,P_I_85610,233,LAB - CHEM & HEMOTOLOGY,16,MISC DIAG THERAPEUTIC,85610,PROTHROMBIN TIME,9999-12-31,CPT-4
4,9,85610,P_I_85610,233,LAB - CHEM & HEMOTOLOGY,16,MISC DIAG THERAPEUTIC,85610,PROTHROMBIN TIME,9999-12-31,CPT-4
...,...,...,...,...,...,...,...,...,...,...,...
27090,9,1162,P_I_1162,013,CORNEAL TRANSPLANT,03,OPERATIONS EYE,1162,OTHER LAMELLAR KERATOPLASTY,9999-12-31,ICD-9
27091,9,67801,P_I_67801,019,OTHER PROC EYELID/CONJ/CORNEA,03,OPERATIONS EYE,67801,REMOVE EYELID LESIONS,9999-12-31,CPT-4
27092,10,0DQ10ZZ,P_I_0DQ10ZZ,094,OTHER OR UPPER GI THERAPEUTIC,09,OPER DIGESTIVE SYS,0DQ10ZZ,REPAIR UP ESOPHAGUS OPEN APPROACH,9999-12-31,ICD10
27093,10,0CUM07Z,P_I_0CUM07Z,033,OTHER OR PROCEDURES ON MOUTH A,05,OPER NOSE MOUTH PHARYNX,0CUM07Z,SUPPL PHAR AUTO TISS SUBST OPN APPR,9999-12-31,ICD10


In [78]:
col = [ 'proc', 'mod_proc_code', 'proc_desc']
merged2 = merged2[col]
proc_final_desc = merged2.drop_duplicates()
proc_final_desc

,proc,mod_proc_code,proc_desc
0,83735,P_I_83735,ASSAY OF MAGNESIUM
3,85610,P_I_85610,PROTHROMBIN TIME
6,82248,P_I_82248,"BILIRUBIN, DIRECT"
9,80053,P_I_80053,COMPREHEN METABOLIC PANEL
12,36415,P_I_36415,ROUTINE VENIPUNCTURE
...,...,...,...
27090,1162,P_I_1162,OTHER LAMELLAR KERATOPLASTY
27091,67801,P_I_67801,REMOVE EYELID LESIONS
27092,0DQ10ZZ,P_I_0DQ10ZZ,REPAIR UP ESOPHAGUS OPEN APPROACH
27093,0CUM07Z,P_I_0CUM07Z,SUPPL PHAR AUTO TISS SUBST OPN APPR


In [79]:
#'DIAGNOSIS_ID','cat','description'
proc_final_desc['cat'] = 'proc'
proc_final_desc= proc_final_desc.drop(columns='proc')
proc_final_desc = proc_final_desc.rename(columns = {'mod_proc_code': 'DIAGNOSIS_ID', 'proc_desc': 'description'})
col = ['DIAGNOSIS_ID','cat','description']
proc_final_desc = proc_final_desc[col]
proc_final_desc

/tmp/ipykernel_3486740/1541561872.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proc_final_desc['cat'] = 'proc'


,DIAGNOSIS_ID,cat,description
0,P_I_83735,proc,ASSAY OF MAGNESIUM
3,P_I_85610,proc,PROTHROMBIN TIME
6,P_I_82248,proc,"BILIRUBIN, DIRECT"
9,P_I_80053,proc,COMPREHEN METABOLIC PANEL
12,P_I_36415,proc,ROUTINE VENIPUNCTURE
...,...,...,...
27090,P_I_1162,proc,OTHER LAMELLAR KERATOPLASTY
27091,P_I_67801,proc,REMOVE EYELID LESIONS
27092,P_I_0DQ10ZZ,proc,REPAIR UP ESOPHAGUS OPEN APPROACH
27093,P_I_0CUM07Z,proc,SUPPL PHAR AUTO TISS SUBST OPN APPR


## Medications

In [81]:
f ='mod_drug'
ot_path = '../../Data_V3/'
med = pd.read_csv(ot_path+f+'.csv', low_memory = False)
med

,patid,encounter_id,mod_gen_drug
0,560499200082764,560499200082764_2017-06-13,M_tacrolimus
1,560499200082764,560499200082764_2017-06-13,M_tacrolimus
2,560499200082764,560499200082764_2017-06-13,M_mycophenolate mofetil
3,560499200082764,560499200082764_2017-07-14,M_tacrolimus
4,560499200082764,560499200082764_2017-07-14,M_mycophenolate mofetil
...,...,...,...
4687073,560499899999890,560499899999890_2010-05-19,M_clonidine hcl
4687074,560499899999890,560499899999890_2010-05-19,M_pantoprazole sodium
4687075,560499899999890,560499899999890_2010-05-19,M_metoprolol tartrate
4687076,560499899999890,560499899999890_2010-05-24,M_alendronate sodium


In [82]:
med_final_desc = med [['patid','mod_gen_drug']]
med_final_desc['cat'] = 'drug'
med_final_desc['description'] = med_final_desc['mod_gen_drug']
#med_final_desc = med['mod_gen_drug']
med_final_desc= med_final_desc.drop(columns='patid')
med_final_desc = med_final_desc.drop_duplicates()
med_final_desc 

,mod_gen_drug,cat,description
0,M_tacrolimus,drug,M_tacrolimus
2,M_mycophenolate mofetil,drug,M_mycophenolate mofetil
18,M_flu vac qs 17-18(4yr up)cel/pf,drug,M_flu vac qs 17-18(4yr up)cel/pf
19,M_imiquimod,drug,M_imiquimod
20,M_montelukast sodium,drug,M_montelukast sodium
...,...,...,...
4637886,M_gel base no41,drug,M_gel base no41
4640281,M_gelatin capsules (empty),drug,M_gelatin capsules (empty)
4641946,M_secretin acetate (human),drug,M_secretin acetate (human)
4653491,M_dm/pe/acetaminoph/diphenhydram,drug,M_dm/pe/acetaminoph/diphenhydram


In [83]:
med_final_desc = med_final_desc.rename(columns = {'mod_gen_drug': 'DIAGNOSIS_ID'})
col = ['DIAGNOSIS_ID','cat','description']
med_final_desc = med_final_desc[col]
med_final_desc 

,DIAGNOSIS_ID,cat,description
0,M_tacrolimus,drug,M_tacrolimus
2,M_mycophenolate mofetil,drug,M_mycophenolate mofetil
18,M_flu vac qs 17-18(4yr up)cel/pf,drug,M_flu vac qs 17-18(4yr up)cel/pf
19,M_imiquimod,drug,M_imiquimod
20,M_montelukast sodium,drug,M_montelukast sodium
...,...,...,...
4637886,M_gel base no41,drug,M_gel base no41
4640281,M_gelatin capsules (empty),drug,M_gelatin capsules (empty)
4641946,M_secretin acetate (human),drug,M_secretin acetate (human)
4653491,M_dm/pe/acetaminoph/diphenhydram,drug,M_dm/pe/acetaminoph/diphenhydram


## Combining All Descriptions

In [84]:
desc = pd.concat([diag_final_desc ,proc_final_desc ,med_final_desc ])

In [85]:
desc

,DIAGNOSIS_ID,cat,description
0,D_10_Z944,diag,LIVER TRANSPLANT STATUS
1,D_10_Z4823,diag,ENC AFTERCARE FLW LIVER TRANSPLANT
2,D_10_E7223,diag,CITRULLINEMIA
3,D_10_Z48298,diag,ENC AFTERCARE FLW OTH ORGN TRANSPL
4,D_10_Z23,diag,ENCOUNTER FOR IMMUNIZATION
...,...,...,...
4637886,M_gel base no41,drug,M_gel base no41
4640281,M_gelatin capsules (empty),drug,M_gelatin capsules (empty)
4641946,M_secretin acetate (human),drug,M_secretin acetate (human)
4653491,M_dm/pe/acetaminoph/diphenhydram,drug,M_dm/pe/acetaminoph/diphenhydram


In [87]:
ot_path = '../../Data_V3/'
desc.to_csv(ot_path+'descriptionsV3.csv',index=False)